<a href="https://colab.research.google.com/github/Devsaa05/Inter_2-incubitu/blob/master/Week7/week_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Week 7 Code!

In [ ]:
!pip install face_recognition
import face_recognition
import cv2

# This is a demo of running face recognition on a video file and saving the results to a new video file.
#
# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Open the input movie file
input_movie = cv2.VideoCapture("/content/y2mate.com - Trump's Tiny Hands_fY2jTTPZFnc_360p.mp4")
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))

# Create an output movie file (make sure resolution/frame rate matches input video!)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_movie = cv2.VideoWriter('output.avi', fourcc, 29.97, (640, 360))

# Load some sample pictures and learn how to recognize them.
lmm_image = face_recognition.load_image_file("/content/donald.jpg")
lmm_face_encoding = face_recognition.face_encodings(lmm_image)[0]

al_image = face_recognition.load_image_file("/content/elon.jpg")
al_face_encoding = face_recognition.face_encodings(al_image)[0]

known_faces = [
    lmm_face_encoding,
    al_face_encoding
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
frame_number = 0

while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)

        # If you had more than 2 faces, you could make this logic a lot prettier
        # but I kept it simple for the demo
        name = None
        if match[0]:
            name = "donald"
        elif match[1]:
            name = "elon"

        face_names.append(name)

    # Label the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if not name:
            continue

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    # Write the resulting image to the output video file
    print("Writing frame {} / {}".format(frame_number, length))
    output_movie.write(frame)

# All done!
input_movie.release()
cv2.destroyAllWindows()

     |████████████████████████████████| 100.2MB 72kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=60db3792834756bb4593c706c63d1cb08949327c66f51a6c3408f2e1f6db6dcd
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models
Writing frame 1 / 2109
Writing frame 2 / 2109
Writing frame 3 / 2109
Writing frame 4 / 2109
Writing frame 5 / 2109
Writing frame 6 / 2109
Writing frame 7 / 2109
Writing frame 8 / 2109
Writing frame 9 / 2109
Writing frame 10 / 2109
Writing frame 11 / 2109
Writing frame 12 / 2109
Writing frame 13 / 2109
Writing frame 14 / 2109
Writing frame 15 / 2109
Writing frame 16 / 2109
Writing frame 17 / 2109
Writing frame 18 / 2109
Writing frame 19 / 2109
Writing frame 20 / 2109
Writing frame 21 / 2109
Writing frame 22 / 2109
Writing frame 23 / 2109
Writing frame 24 / 2109
Writing frame 25 / 2109
Writing fra

In [2]:
!pip install face_recognition
from imutils.video import VideoStream
import imutils
import pickle 
import argparse
import face_recognition
import time
import cv2

ap = argparse.ArgumentParser()
ap.add_argument('-e', '--encodings', required= True, help='/Users/hujjatzaidi/Desktop/currentProjects')
ap.add_argument('-d', '--detection_method', type=str, default= 'hog', help='Detection Method is : hog or cnn')
args = vars(ap.parse_args())

print('Loading encodings')
data = pickle.loads(open(args['encodings'], 'rb').read())
print('Video stream started')
vs = VideoStream(0).start()
writer = None
time.sleep(2.0)

while True:
	frame = vs.read()
	rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	rgb = imutils.resize(frame, width = 750)
	r = frame.shape[1] / float(rgb.shape[1])

	boxes = face_recognition.face_locations(rgb, model= args['detection_method'])
	encodings = face_recognition.face_encodings(rgb, boxes)
	names = []

	for encoding in encodings:
		matches = face_recognition.compare_faces(data['encodings'], encoding)
		name = 'unknown'

		if True in matches:
			matchedIdxs = [i for (i, b) in enumerate(matches) if b]
			counts = {}
 
			for i in matchedIdxs:
				name = data["names"][i]
				counts[name] = counts.get(name, 0) + 1
			name = max(counts, key=counts.get)
		
		names.append(name)

	for ((top, right, bottom, left), name) in zip(boxes, names):
		top = int(top * r)
		right = int(right * r)
		bottom = int(bottom * r)
		left = int(left * r)
 
		cv2.rectangle(frame, (left, top), (right, bottom),
			(0, 255, 0), 2)
		y = top - 15 if top - 15 > 15 else top + 15
		cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
			0.75, (0, 255, 0), 2)

	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF
 
	if key == ord("q"):
		break

cv2.destroyAllWindows()
vs.stop()
 
if writer is not None:
	writer.release()

     |████████████████████████████████| 100.2MB 73kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=19d5ca9dfae476773ac95e537dcbc3d19b1cf18fc54e914f54d75fa91a5e49b2
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


usage: ipykernel_launcher.py [-h] -e ENCODINGS [-d DETECTION_METHOD]
ipykernel_launcher.py: error: the following arguments are required: -e/--encodings


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


This is code written after week 5 review meeting just to read the video and save indivuial frame!

In [ ]:
!pip install face_recognition
import cv2
import face_recognition 
# Opens the Video file
cap= cv2.VideoCapture('/content/face.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('kang'+str(i)+'.jpg',frame)
    i+=1
    list = [kang'+str(i)+'.jpg] 
for index in range(len(list)): 
    


picture_of_me = face_recognition.load_image_file("/content/donald.jpg")
my_face_encoding = face_recognition.face_encodings(picture_of_me)[0]

# my_face_encoding now contains a universal 'encoding' of my facial features that can be compared to any other picture of a face!

unknown_picture = face_recognition.load_image_file(list[index])
unknown_face_encoding = face_recognition.face_encodings(unknown_picture)[0]

# Now we can see the two face encodings are of the same person with `compare_faces`!

results = face_recognition.compare_faces([my_face_encoding], unknown_face_encoding)

if results[0] == True:
    print("Both are same person!")
else:
    print("Both person are different!") 
cap.release()
cv2.destroyAllWindows()

SyntaxError: ignored

# Now this is to detect face from single image using CNN

Using Haar cascade method

In [ ]:
import cv2


# Load the cascade
face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
# Read the input image
img = cv2.imread('/content/DSC_0374.JPG')
img_1 = img 
# Convert into grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
# Draw rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

# Display the output
cv2.imwrite('img.jpg', img_1)
cv2.waitKey()

-1

In [ ]:
!pip install face_recognition
# import libraries
import cv2
import face_recognition

# Get a reference to webcam 
video_capture = cv2.VideoCapture("/content/HD CCTV Camera video 3MP 4MP iProx CCTV HDCCTVCameras.net retail store.mp4")

# Initialize variables
face_locations = []

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)

    # Display the results
    for top, right, bottom, left in face_locations:
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

    # Display the resulting image
    
    cv2.imwrite('Video.mp4', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

error: ignored

In [ ]:
!pip install face_recognition
import cv2
import face_recognition

video_capture = cv2.VideoCapture('/content/face.mp4')

face_locations = []

while True:
 # Grab a single frame of video
 ret, frame = video_capture.read()# Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
 rgb_frame = frame[:, :, ::-1]# Find all the faces in the current frame of video
 face_locations = face_recognition.face_locations(rgb_frame)# Display the results
 for top, right, bottom, left in face_locations:
 # Draw a box around the face
  cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)# Display the resulting image
 cv2.imshow("Video", frame)# Hit ‘q’ on the keyboard to quit!
 if cv2.waitKey(1) & 0xFF == ord("q"):
  break

     |████████████████████████████████| 100.2MB 42kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=51d9fc0fe651c30f17bf31196306e902d9615c24fc8cf0cb1ebcd37bed796504
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


TypeError: ignored